<a href="https://colab.research.google.com/github/jenieto/pollen-classification/blob/master/Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montamos Google Drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Extract data
!mkdir -p /data
!tar xvzf "/content/drive/My Drive/Datasets/anuka1200.tar.gz" --directory /data

In [ ]:
# Create datasets
import tensorflow as tf

training_size = 0.8
validation_size = 0.2
image_height = 50
image_width = 50
image_channels = 3

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/data/anuka1200',
    subset="training",
    validation_split=training_size,
    seed=123,
    image_size=(image_height, image_width))

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/data/anuka1200',
    subset="validation",
    validation_split=validation_size,
    seed=123,
    image_size=(image_height, image_width))

class_names = train_dataset.class_names
print('Class Names', class_names)

In [ ]:
# Explore data
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [45]:
# Create model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=image_size),
  layers.Conv2D(32, (3, 3), activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(2, activation='sigmoid')
])

In [ ]:
# Compile model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Train model
epochs=10
history = model.fit(
  train_dataset,
  validation_data=validation_dataset,
  epochs=epochs
)